### Calculate and Plot Fermi beta angle

### <span style="color:red; font-family:Georgia;">Robert Cameron, May 2017</span>

Based on the python implementation of SGP4 orbit propagator
available at https://pypi.python.org/pypi/sgp4/

In [1]:
# set the date range limits to calculate the beta angle for Fermi
start = "2018-01-01" 
stop = "2019-01-01"

In [2]:
# set the output directory for output text files and plots
outdir = "/Users/rac/Documents/GLAST/ISOC/LATmetrics/beta/"

In [3]:
import numpy as np
import re
import urllib
from math import *
%matplotlib notebook  # it seems that this line must be before the matplotlib imports
import matplotlib.pyplot as plt, mpld3
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
import sys
from os.path import expanduser
HOME = expanduser("~")
sys.path.append(HOME+'/Documents/GLAST/sgp4-1.4')
from sgp4.earth_gravity import wgs84
from sgp4.io import twoline2rv
from sgp4.ext import jday, invjday, days2mdhms
import datetime as dt     
today = str(dt.date.today())

UsageError: unrecognized arguments: # it seems that this line must be before the matplotlib imports


In [ ]:
# get latest TLE (Two Line Elements) for Fermi orbit

webTLE = "http://celestrak.com/NORAD/elements/science.txt"
lines = urllib.request.urlopen(webTLE).readlines()
regex = re.compile(b'(1|2) 33053')
tle = [x.decode().replace('\r\n', '') for x in lines if regex.match(x)]
tle

In [ ]:
# use WGS84 Earth Coordinate System in TLE propagation
satellite = twoline2rv(tle[0], tle[1], wgs84)

In [ ]:
# The Fermi MOC and FSSC have the convention that the start of January 1 UTC = day-of-year (DOY) 1.0
(y0,m0,d0) = map(int, start.split("-"))
jd0 = jday(y0,m0,d0,0,0,0)
doy0 = jd0 - jday(y0,1,0,0,0,0)
# omit the following line to have the beta angle time scale agree with the MOC/FOT and the FSSC
#doy0 -= 1       ##### include this line to have the start of the year at Day-Of-Year = 0.0
(y1,m1,d1) = map(int, stop.split("-"))
ndays = int(jday(y1,m1,d1,0,0,0)  - jd0)
(y0, doy0, ndays)

In [ ]:
# propagate Fermi orbit and calculate beta angle at 1-hour intervals
# save data to a text file

oformat = "%4d %02d %02d %02d %02d %02d %12.5f %12.6f %12.6f %12.6f %8.3f\n"
fo = open(outdir+"beta_"+today+".txt", "w")
fo.write("Year MonDay H M  S     DOY     X(km)      Y(km)       Z(km)         beta\n"
fdoypl = []
betapl = []
for day in range(ndays):
    for hour in range(24):
# add a small amount to the day number to ensure it is always a fraction of a second past the hour
        dinc = day + hour/24.0 + 1.0e-9
        jd = jd0 + dinc
        fdoy = doy0 + dinc
        (y,mon,d,h,m,s) = invjday(jd)    # need Julian Day for the Sun ephemeris
        s = int(s)
        fdoypl.append(fdoy)
        pos, vel = satellite.propagate(y,mon,d,h,m,s)  # this is the key line to specify the date
        pole = np.cross(pos,vel)
        polelen = np.linalg.norm(pole)
# calculate Sun coordinates. See (for example): https://en.wikipedia.org/wiki/Position_of_the_Sun
        n = jd - 2451545.0
        L = (280.460 + 0.9856474 * n) % 360
        g = radians((357.528 + 0.9856003 * n) % 360)
        lamda = radians(L + 1.915 * sin(g) + 0.020 * sin(2*g))
        epsilon = radians(23.439 - 0.0000004 * n)
        Xsun = cos(lamda)
        Ysun = cos(epsilon) * sin(lamda)
        Zsun = sin(epsilon) * sin(lamda)
        dotprod = np.dot([Xsun,Ysun,Zsun], pole)
        beta = degrees(np.arcsin(dotprod/polelen))
        betapl.append(beta)
        fo.write(oformat % (y, mon, d, h, m, s, fdoy, pos[0], pos[1], pos[2], beta))
fo.close()

In [ ]:
# set minor ticks on plot
minorLocatorx = AutoMinorLocator(10)
minorLocatory = AutoMinorLocator(10)

#plt.rc('font', size = 12)
fig, ax = plt.subplots(figsize = [11, 8.5])

ax.set_ylim(-50,50)
ax.grid('on',which='major', axis='both', linewidth=0.5, color='k')
ax.grid('on',which='minor', axis='both',linestyle = 'dashed')
ax.plot(fdoypl, betapl, label = 'Beta angle', color = 'b')
ax.set_xlabel(str(y0)+' Day of Year')
ax.set_ylabel('beta (degrees)', color = 'b')
ax.set_title('Fermi Beta Angle')
ax.xaxis.set_minor_locator(minorLocatorx)
ax.yaxis.set_minor_locator(minorLocatory)
xminmax = [fdoypl[0],fdoypl[-1]]

ax.plot(xminmax,[-45,-45], color = 'r')
ax.plot(xminmax,[45,45], color = 'r')

mpld3.enable_notebook()

In [ ]:
plt.savefig(outdir+'beta_'+today+'.png')